In [10]:
import requests
import json
import datetime
import pandas as pd
import pytz
from tabulate import tabulate

class FastGenie:
    
    def __init__(self, location, start_date):
        street, city, state, zipcode = location.split(',')
        self.street = street.strip()
        self.city = city.strip()
        self.state = state.strip()
        self.zipcode = zipcode.strip()
        self.start_date = start_date
        self.description = "This is very simple tool to generate a time table for nerdy Baha'is during the fast."
        
    def geo_api(self):
        GEO_API = 'https://geocoding.geo.census.gov/geocoder/geographies/address?'
        api_out = requests.get(GEO_API, params={
            'returntype': 'geographies',
            'searchtype': 'address',
            'street': self.street,
            'city': self.city,
            'state': self.state,
            'zip': self.zipcode,
            'benchmark': 'Public_AR_Current',
            'vintage': 'Current_Current',
            'layers': 14,
            'format': 'json',}).json()['result']
        
        lat = api_out['addressMatches'][0]['coordinates']['y']
        lng = api_out['addressMatches'][0]['coordinates']['x']
        
        return lat,lng
    
    def get_sun_schedule(self):
        lat,lng = self.geo_api()
        SUNRISE_API = 'https://api.sunrise-sunset.org/json?'
        series_date = pd.date_range(self.start_date, freq='D', periods=21)
        datelist = series_date.astype(str)
        
        output = list()
        
        # Easier to loop to collect the date, too
        for date in datelist:
            api_out = requests.get(SUNRISE_API, params={'lat': lat,
                                                        'lng': lng,
                                                        'date': date,
                                                        'formatted': 0}).json()['results']
            api_out['date'] = date
            api_out['day_of_week'] = datetime.datetime.strptime(date, '%Y-%m-%d').strftime('%A')
            output.append(api_out)
        return output

    def get_time_table(self):
        good_data = self.get_sun_schedule()
        df = pd.DataFrame(good_data)
        df = df[['day_of_week','date','sunrise','sunset']]
        
        # Set timezone
        tz = pytz.timezone('US/Pacific')
        df['sunrise'] = pd.to_datetime(df['sunrise']).dt.tz_convert('US/Pacific')
        df['sunset'] = pd.to_datetime(df['sunset']).dt.tz_convert('US/Pacific')
        
        # Beautify
        df['sunrise'] = df['sunrise'].apply(lambda x: x.strftime('%I:%M:%S %p'))
        df['sunset'] = df['sunset'].apply(lambda x: x.strftime('%I:%M:%S %p'))
        
        return df

## Enter your address and start date, and run this baby!

In [11]:
YOUR_ADDRESS = '268 Coronet Dr,Los Gatos,CA,95032'
FIRST_DAY_OF_FAST = '2023-03-02'
FastPrinter = FastGenie(YOUR_ADDRESS,FIRST_DAY_OF_FAST)
print(tabulate(FastPrinter.get_time_table(), headers='keys', tablefmt='psql', showindex=False))
# FastPrinter.get_time_table()

+---------------+------------+-------------+-------------+
| day_of_week   | date       | sunrise     | sunset      |
|---------------+------------+-------------+-------------|
| Thursday      | 2023-03-02 | 06:35:53 AM | 06:03:50 PM |
| Friday        | 2023-03-03 | 06:34:29 AM | 06:04:48 PM |
| Saturday      | 2023-03-04 | 06:33:06 AM | 06:05:46 PM |
| Sunday        | 2023-03-05 | 06:31:41 AM | 06:06:44 PM |
| Monday        | 2023-03-06 | 06:30:16 AM | 06:07:41 PM |
| Tuesday       | 2023-03-07 | 06:28:50 AM | 06:08:38 PM |
| Wednesday     | 2023-03-08 | 06:27:24 AM | 06:09:35 PM |
| Thursday      | 2023-03-09 | 06:25:58 AM | 06:10:32 PM |
| Friday        | 2023-03-10 | 06:24:31 AM | 06:11:28 PM |
| Saturday      | 2023-03-11 | 06:23:03 AM | 06:12:24 PM |
| Sunday        | 2023-03-12 | 07:21:35 AM | 07:13:20 PM |
| Monday        | 2023-03-13 | 07:20:07 AM | 07:14:16 PM |
| Tuesday       | 2023-03-14 | 07:18:39 AM | 07:15:12 PM |
| Wednesday     | 2023-03-15 | 07:17:10 AM | 07:16:07 PM